# 第7章: 機械学習

本章では、[Stanford Sentiment Treebank (SST)](https://nlp.stanford.edu/sentiment/) データセットを用い、評判分析器（ポジネガ分類器）を構築する。ここでは処理を簡略化するため、[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されているSSTデータセットを用いる。


In [1]:
%%capture
!pip install scikit-learn
!pip install numpy
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

## 60. データの入手・整形

GLUEのウェブサイトから[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)データセットを取得せよ。学習データ（`train.tsv`）と検証データ（`dev.tsv`）のぞれぞれについて、ポジティブ (1) とネガティブ (0) の事例数をカウントせよ。

In [13]:
import pandas as pd

# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', header=None)
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', header=None)

train_pos_count = (train_df[1] == '1').sum()
train_neg_count = (train_df[1] == '0').sum()
dev_pos_count = (dev_df[1] == '1').sum()
dev_neg_count = (dev_df[1] == '0').sum()

print("Train: Positive =", train_pos_count, ", Negative =", train_neg_count)
print("Valid: Positive =", dev_pos_count, ", Negative =", dev_neg_count)
train_df.head()

Train: Positive = 37569 , Negative = 29780
Valid: Positive = 444 , Negative = 428


,0,1
0,sentence,label
1,hide new secretions from the parental units,0
2,"contains no wit , only labored gags",0
3,that loves its characters and communicates som...,1
4,remains utterly satisfied to remain the same t...,0


## 61. 特徴ベクトル

Bag of Words (BoW) に基づき、学習データ（`train.tsv`）および検証データ（`dev.tsv`）のテキストを特徴ベクトルに変換したい。ここで、ある事例のテキストの特徴ベクトルは、テキスト中に含まれる単語（スペース区切りのトークン）の出現頻度で構成する。例えば、"too loud , too goofy"というテキストに対応する特徴ベクトルは、以下のような辞書オブジェクトで表現される。

```python
{'too': 2, 'loud': 1, ',': 1, 'goofy': 1}
```

各事例はテキスト、特徴ベクトル、ラベルを格納した辞書オブジェクトでまとめておく。例えば、先ほどの"too loud , too goofy"に対してラベル"0"（ネガティブ）が付与された事例は、以下のオブジェクトで表現される。

```python
{'text': 'too loud , too goofy', 'label': '0', 'feature': {'too': 2, 'loud': 1, ',': 1, 'goofy': 1}}
```

学習データと検証データの各事例を上記のような辞書オブジェクトに変換したうえで、学習データと検証データのそれぞれを、辞書オブジェクトのリストとして表現せよ。さらに、学習データの最初の事例について、正しく特徴ベクトルに変換できたか、目視で確認せよ。

In [24]:
from collections import Counter

def text_to_features(text):
    # スペース区切りで単語を分割
    words = text.split()
    # 単語の出現頻度をカウント
    feature_vector = dict(Counter(words))
    return feature_vector

# 学習データと検証データのリストを作成する関数
def data_to_object(data_df):
    objects = []
    for i in range(1,len(data_df)):
      text = data_df.iloc[i, 0]   # 1列目（0-indexed、つまり'sentence'列）
      label = data_df.iloc[i, 1]

      feature = text_to_features(text)
      objects.append({'text': text, 'label': label, 'feature': feature})
    return objects

train_objects = data_to_object(train_df)
dev_objects = data_to_object(dev_df)

[{'text': 'hide new secretions from the parental units ',
  'label': '0',
  'feature': {'hide': 1,
   'new': 1,
   'secretions': 1,
   'from': 1,
   'the': 1,
   'parental': 1,
   'units': 1}},
 {'text': 'contains no wit , only labored gags ',
  'label': '0',
  'feature': {'contains': 1,
   'no': 1,
   'wit': 1,
   ',': 1,
   'only': 1,
   'labored': 1,
   'gags': 1}},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature ',
  'label': '1',
  'feature': {'that': 1,
   'loves': 1,
   'its': 1,
   'characters': 1,
   'and': 1,
   'communicates': 1,
   'something': 1,
   'rather': 1,
   'beautiful': 1,
   'about': 1,
   'human': 1,
   'nature': 1}},
 {'text': 'remains utterly satisfied to remain the same throughout ',
  'label': '0',
  'feature': {'remains': 1,
   'utterly': 1,
   'satisfied': 1,
   'to': 1,
   'remain': 1,
   'the': 1,
   'same': 1,
   'throughout': 1}},
 {'text': 'on the worst revenge-of-the-nerds clichés the filmmakers coul

In [25]:
# 1. 全語彙を収集（順序付きで）
def build_vocab(data_objects):
    vocab = set()
    for obj in data_objects:
        vocab.update(obj['feature'].keys())
    return sorted(vocab)  # ソートして順序を固定

# 2. 特徴ベクトルに変換（語彙リストに基づいて）
def convert_to_vector(data_objects, vocab):
    vectors = []
    for obj in data_objects:
        vector = [obj['feature'].get(word, 0) for word in vocab]
        vectors.append(vector)
    return vectors

In [26]:
vocab = build_vocab(train_objects)
train_vectors = convert_to_vector(train_objects, vocab)

# 例: 最初のベクトルを確認
print("語彙数:", len(vocab))
print("最初の特徴ベクトル:", train_vectors[0])


語彙数: 14816
最初の特徴ベクトル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 62. 学習

61で構築した学習データの特徴ベクトルを用いて、ロジスティック回帰モデルを学習せよ。

## 63. 予測

学習したロジスティック回帰モデルを用い、検証データの先頭の事例のラベル（ポジネガ）を予測せよ。また、予測されたラベルが検証データで付与されていたラベルと一致しているか、確認せよ。

## 64. 条件付き確率

学習したロジスティック回帰モデルを用い、検証データの先頭の事例を各ラベル（ポジネガ）に分類するときの条件付き確率を求めよ。

## 65. テキストのポジネガの予測

与えられたテキストのポジネガを予測するプログラムを実装せよ。例えば、テキストとして"the worst movie I 've ever seen"を与え、ロジスティック回帰モデルの予測結果を確認せよ。


## 66. 混同行列の作成

学習したロジスティック回帰モデルの検証データにおける混同行列（confusion matrix）を求めよ。

## 67. 精度の計測

学習したロジスティック回帰モデルの正解率、適合率、再現率、F1スコアを、学習データおよび検証データ上で計測せよ。

## 68. 特徴量の重みの確認

学習したロジスティック回帰モデルの中で、重みの高い特徴量トップ20と、重みの低い特徴量トップ20を確認せよ。

## 69. 正則化パラメータの変更

ロジスティック回帰モデルを学習するとき、正則化の係数（ハイパーパラメータ）を調整することで、学習時の適合度合いを制御できる。正則化の係数を変化させながらロジスティック回帰モデルを学習し、検証データ上の正解率を求めよ。実験の結果は、正則化パラメータを横軸、正解率を縦軸としたグラフにまとめよ。